In [20]:
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import math
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, KBinsDiscretizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from time import sleep

pd.set_option('display.float_format', lambda x: '%.3f' % x)

sns.set()

In [22]:
def generaPath(tipo, nombre):
    prepath = str(os.getcwd())
    #print(prepath)
    if (tipo == "I"):
        path = prepath + '\\' + nombre
    else :
        path = prepath + '\\generados\\' + nombre
    return path

In [24]:
df_cq_tr = pd.read_csv(generaPath('O', 'df_cq_tr.csv'))
df_cq_re = pd.read_csv(generaPath('O', 'df_cq_re.csv'))

df_cc_tr = pd.read_csv(generaPath('O', 'df_cc_tr.csv'))
df_cc_re = pd.read_csv(generaPath('O', 'df_cc_re.csv'))

df_ct_tr = pd.read_csv(generaPath('O', 'df_ct_tr.csv'))
df_ct_re = pd.read_csv(generaPath('O', 'df_ct_re.csv'))

df_cd_tr = pd.read_csv(generaPath('O', 'df_cd_tr.csv'))
df_cd_re = pd.read_csv(generaPath('O', 'df_cd_re.csv'))

df_cs_tr = pd.read_csv(generaPath('O', 'df_cs_tr.csv'))
df_cs_re = pd.read_csv(generaPath('O', 'df_cs_re.csv'))

df_cs_tr_sb = pd.read_csv(generaPath('O', 'df_cs_tr_sb.csv'))
df_cs_re_sb = pd.read_csv(generaPath('O', 'df_cs_re_sb.csv'))



In [26]:
df_cs_tr_sb

,conteo,tupla,X,L,R,M,T,P,Pr,PrX,PrL,PrM
0,28,4,15,18,3,5|6|1,18|5|6|3|18|11|9|2|3|23|137|1|5|4|13|19|28|20|...,0.062,0.687,1.000,1.000,0.062
1,26,8,16,19,3,3|4,19|3|4|10|28|7|3|42|40|18|13|17|3|78|4|8|17|20...,0.058,0.686,1.000,1.000,0.058
2,43,11,10,7,3,1,7|7|2|1|4|17|54|19|16|6|19|15|3|10|4|9|1|1|3|2...,0.095,0.698,1.000,1.000,0.095
3,36,2,12,8,4,2,8|7|12|16|3|8|2|11|37|19|17|3|17|3|3|5|2|8|4|3...,0.080,0.693,1.000,1.000,0.080
4,31,3,14,20,6,20|1,20|20|9|6|21|27|4|51|3|27|20|1|4|39|12|2|1|2|6...,0.069,0.690,1.000,1.000,0.069
5,28,5,15,6,9,6,6|5|1|22|10|3|11|41|2|16|1|6|14|8|8|28|34|12|5...,0.062,0.687,1.000,1.000,0.062
6,23,14,18,9,9,1,9|51|17|11|24|29|3|13|46|29|10|1|1|3|14|41|4|2...,0.051,0.684,1.000,1.000,0.051
7,32,16,13,4,9,6,4|9|24|5|6|14|7|12|5|24|5|7|6|6|43|33|20|1|17|...,0.071,0.690,1.000,1.000,0.071
8,38,13,11,1,10,3,1|4|5|18|3|2|18|6|16|5|4|5|7|3|12|30|20|13|6|1...,0.084,0.695,1.000,1.000,0.084
9,23,6,18,3,15,3|16|10|5|50,3|60|16|10|32|5|23|6|50|41|7|50|26|1|16|9|27|1...,0.051,0.684,1.000,1.000,0.051


In [28]:
def cayo_antestetos(conteos, df):
    media = []
    modaa = []
    lasts = []
    delta = []  
    resta = []
    for index, row in conteos.iterrows():
        sbs = str(row[1])
        splitedtupla= unsplited_tupla(sbs)
        sbs1 = [str(splitedtupla[0])]
        sbs2 = [str(splitedtupla[1])]
        sbs3 = [str(splitedtupla[2])]
        sbs4 = [str(splitedtupla[3])]
        sbs5 = [str(splitedtupla[4])]
        dftx = df[(df.n1.isin(sbs1) | df.n2.isin(sbs1) |df.n3.isin(sbs1) | df.n4.isin(sbs1) |df.n5.isin(sbs1)) 
                & (df.n1.isin(sbs2) | df.n2.isin(sbs2) |df.n3.isin(sbs2) | df.n4.isin(sbs2) |df.n5.isin(sbs2)) 
                & (df.n1.isin(sbs3) | df.n2.isin(sbs3) |df.n3.isin(sbs3) | df.n4.isin(sbs3) |df.n5.isin(sbs3))
                & (df.n1.isin(sbs4) | df.n2.isin(sbs4) |df.n3.isin(sbs4) | df.n4.isin(sbs4) |df.n5.isin(sbs4))
                & (df.n1.isin(sbs5) | df.n2.isin(sbs5) |df.n3.isin(sbs5) | df.n4.isin(sbs5) |df.n5.isin(sbs5)) ]
        #print(dftx)
        df_filt = dftx.iloc[0::1,[0]]
        df_tupla = calcular_deltas_verticales(df_filt)
        lasts.append(df_tupla[0])
        media.append(calc_media(df_tupla))
        resta.append(abs(int(df_tupla[0])- int(calc_media(df_tupla))))
        df_moda = calc_moda(df_tupla)
        pal = '|'.join([str(m) for m in df_moda]) if len(df_moda) > 1  else str(df_moda[0])
        modaa.append(pal)
        palabra = '|'.join([str(e) for e in df_tupla]) 
        delta.append(palabra)
    dft = conteos
    dft['X'] = media
    dft['L'] = lasts
    dft['R'] = resta
    dft['M'] = modaa
    df_filt2 = dft.iloc[0::1,[0]]
    #print("df_filt2", df_filt2)
    df_pesos = sumarcol_peso(df_filt2)
    #print("valor de df_pesos ", df_pesos)
    dft['T'] = delta
    dft['P'] = df_pesos
    proba, proba1,proba2,proba3 = how_hot_is(dft, df_tr_fu_filt_delta)
    dft['Pr']= proba
    dft['PrX'] = proba1
    dft['PrL'] = proba2
    dft['PrM'] = proba3
    return dft

#### Introduzca datos para validar cuartetos y quintetos

In [30]:
def construir_quinteto(n1,n2,n3,n4,n5):
    salida = str(n1) + "|" + str(n2) + "|" + str(n3) + "|" + str(n4) + "|" + str(n5)
    return salida

In [32]:
def construir_cuarteto(n1,n2,n3,n4):
    salida = str(n1) + "|" + str(n2) + "|" + str(n3) + "|" + str(n4)
    return salida

In [34]:
def construir_trio(n1,n2,n3):
    salida = str(n1) + "|" + str(n2) + "|" + str(n3)
    return salida

In [36]:
def all_cuartetos(n1,n2,n3,n4,n5):
    cuarteto1 = construir_cuarteto(n1, n2, n3, n4)
    cuarteto2 = construir_cuarteto(n1, n2, n3, n5)
    cuarteto3 = construir_cuarteto(n1, n2, n4, n5)
    cuarteto4 = construir_cuarteto(n1, n3, n4, n5)
    cuarteto5 = construir_cuarteto(n2, n3, n4, n5)
    return cuarteto1, cuarteto2, cuarteto3, cuarteto4, cuarteto5

In [38]:
def all_trios(n1,n2,n3,n4,n5):
    trio01 = construir_trio(n1,n2,n3)
    trio02 = construir_trio(n1,n2,n4)
    trio03 = construir_trio(n1,n2,n5)
    trio04 = construir_trio(n2,n3,n4)
    trio05 = construir_trio(n2,n3,n5)
    trio06 = construir_trio(n2,n4,n5)
    trio07 = construir_trio(n3,n4,n5)
    trio08 = construir_trio(n1,n3,n4)
    trio09 = construir_trio(n1,n3,n5)
    trio10 = construir_trio(n1,n4,n5)
    return trio01, trio02, trio03, trio04, trio05, trio06, trio07, trio08, trio09, trio10

In [40]:
def ingresar_sorteo_validar():
    n1 = int(input(f"ingresa el primer numero del sorteo: "))
    n2 = int(input(f"ingresa el segundo numero del sorteo: "))
    n3 = int(input(f"ingresa el tercer numero del sorteo: "))
    n4 = int(input(f"ingresa el cuarto numero del sorteo: "))    
    n5 = int(input(f"ingresa el quinto numero del sorteo: "))
    quinteto = construir_quinteto(n1, n2, n3, n4, n5)
    cuarteto1, cuarteto2, cuarteto3, cuarteto4, cuarteto5 = all_cuartetos(n1, n2, n3, n4, n5)
    trio01, trio02, trio03, trio04, trio05, trio06, trio07, trio08, trio09, trio10 = all_trios(n1, n2, n3, n4, n5)
    return quinteto, cuarteto1, cuarteto2, cuarteto3, cuarteto4, cuarteto5, trio01, trio02, trio03, trio04, trio05, trio06, trio07, trio08, trio09, trio10

In [42]:
def find_previous_coincidence(value_to_search, df):
    df_found = df.loc[df['tupla']== value_to_search]
    total_rows = int(df_found.shape[0])
    if total_rows > 0:
        return "Records Found: " + str(total_rows) +' - '+ str(value_to_search)
    else:
        return "No Records Found"

In [44]:
def validar_imprimir_resultados_sorteos(dictionary, propiedad, dfbase, df_to_search):
    dictionary[propiedad] = propiedad
    dictionary
    print(find_previous_coincidence(dfbase, df_to_search))

In [45]:
i = 1
dictionary = {}
while i > 0:
    quinteto, cuarteto1, cuarteto2, cuarteto3, cuarteto4, cuarteto5, trio01, trio02, trio03, trio04, trio05, trio06, trio07, trio08, trio09,         trio10 = ingresar_sorteo_validar()
    validar_imprimir_resultados_sorteos(dictionary, "trios01tr", trio01, df_ct_tr)
    validar_imprimir_resultados_sorteos(dictionary, "trios01re", trio01, df_ct_re) 
    validar_imprimir_resultados_sorteos(dictionary, "trios02tr", trio02, df_ct_tr)
    validar_imprimir_resultados_sorteos(dictionary, "trios02re", trio02, df_ct_re) 
    validar_imprimir_resultados_sorteos(dictionary, "trios03tr", trio03, df_ct_tr)
    validar_imprimir_resultados_sorteos(dictionary, "trios03re", trio03, df_ct_re) 
    validar_imprimir_resultados_sorteos(dictionary, "trios04tr", trio04, df_ct_tr)
    validar_imprimir_resultados_sorteos(dictionary, "trios04re", trio04, df_ct_re) 
    validar_imprimir_resultados_sorteos(dictionary, "trios05tr", trio05, df_ct_tr)
    validar_imprimir_resultados_sorteos(dictionary, "trios05re", trio05, df_ct_re)
    validar_imprimir_resultados_sorteos(dictionary, "trios06tr", trio06, df_ct_tr)
    validar_imprimir_resultados_sorteos(dictionary, "trios06re", trio06, df_ct_re) 
    validar_imprimir_resultados_sorteos(dictionary, "trios07tr", trio07, df_ct_tr)
    validar_imprimir_resultados_sorteos(dictionary, "trios07re", trio07, df_ct_re) 
    validar_imprimir_resultados_sorteos(dictionary, "trios08tr", trio08, df_ct_tr)
    validar_imprimir_resultados_sorteos(dictionary, "trios08re", trio08, df_ct_re) 
    validar_imprimir_resultados_sorteos(dictionary, "trios09tr", trio09, df_ct_tr)
    validar_imprimir_resultados_sorteos(dictionary, "trios09re", trio09, df_ct_re) 
    validar_imprimir_resultados_sorteos(dictionary, "trios10tr", trio10, df_ct_tr)
    validar_imprimir_resultados_sorteos(dictionary, "trios10re", trio10, df_ct_re)
    print("Cuartetos")
    validar_imprimir_resultados_sorteos(dictionary, "cuarteto1tr", cuarteto1, df_cc_tr)
    validar_imprimir_resultados_sorteos(dictionary, "cuarteto1re", cuarteto1, df_cc_re) 
    validar_imprimir_resultados_sorteos(dictionary, "cuarteto2tr", cuarteto2, df_cc_tr)
    validar_imprimir_resultados_sorteos(dictionary, "cuarteto2re", cuarteto2, df_cc_re) 
    validar_imprimir_resultados_sorteos(dictionary, "cuarteto3tr", cuarteto3, df_cc_tr)
    validar_imprimir_resultados_sorteos(dictionary, "cuarteto3re", cuarteto3, df_cc_re) 
    validar_imprimir_resultados_sorteos(dictionary, "cuarteto4tr", cuarteto4, df_cc_tr)
    validar_imprimir_resultados_sorteos(dictionary, "cuarteto4re", cuarteto4, df_cc_re) 
    validar_imprimir_resultados_sorteos(dictionary, "cuarteto5tr", cuarteto5, df_cc_tr)
    validar_imprimir_resultados_sorteos(dictionary, "cuarteto5re", cuarteto5, df_cc_re)
    print("Quintetos")
    validar_imprimir_resultados_sorteos(dictionary, "quintetotr", quinteto,df_cq_tr)
    validar_imprimir_resultados_sorteos(dictionary, "quintetore", quinteto,df_cq_re)
    print("---------")
    i = int(input(f"Ingresa 1 para continuar evaluando sorteos, 0 para parar: "))
    if i < 1:
        break
    


No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
Cuartetos
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
No Records Found
Quintetos
No Records Found
No Records Found
---------


In [46]:
#print(find_previous_coincidence(quinteto, df_cq_tr))

In [47]:
#print(find_previous_coincidence(quinteto, df_cq_re))

In [48]:
#print(find_previous_coincidence(cuarteto1, df_cc_tr))

In [49]:
#print(find_previous_coincidence(cuarteto1, df_cc_re))

In [50]:
#print(find_previous_coincidence(cuarteto2, df_cc_tr))

In [51]:
#print(find_previous_coincidence(cuarteto2, df_cc_re))

In [52]:
#print(find_previous_coincidence(cuarteto3, df_cc_tr))

In [53]:
#print(find_previous_coincidence(cuarteto3, df_cc_re))

In [54]:
#print(find_previous_coincidence(cuarteto4, df_cc_tr))

In [55]:
#print(find_previous_coincidence(cuarteto4, df_cc_re))

In [56]:
#print(find_previous_coincidence(cuarteto5, df_cc_tr))

In [57]:
#print(find_previous_coincidence(cuarteto5, df_cc_re))